In [ ]:
## Code to extract features from images with abnormalities...

In [ ]:
import os
import random
import cv2
import warnings
import torch
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os.path

#from SSIM_PIL import compare_ssim
from PIL import Image

import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger

from sklearn.model_selection import GroupKFold
#from icevision.all import *

warnings.filterwarnings('ignore')
import os

In [ ]:
# Load train csv data file
df = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
print(df.head())
print(df.shape)
# The total number of unique images 
count = pd.Series(os.listdir('../input/vinbigdata-chest-xray-abnormalities-detection/train/'))
print(count.nunique())
print(len(count))

In [ ]:
# Additional features 
df['image_name'] = '../input/vinbigdata-chest-xray-abnormalities-detection/train/' + df.image_id + '.dicom'
df['xd'] = df.x_max - df.x_min
df['yd'] = df.y_max - df.y_min
df['area'] = df.xd*df.yd
df.head()

In [ ]:
print(df['image_id'].nunique())
print(len(df['image_id']))

In [ ]:
tab = pd.DataFrame(df['image_id'].value_counts())
tab.columns = ['count']
tab['image_id'] = tab.index
tab1= tab.reset_index()

In [ ]:
tab1.drop('index', axis =1,  inplace = True)
tab1.head()

In [ ]:
tab.shape

In [ ]:
#print(df[df['image_id'] == tab1.image_id[0]])

In [ ]:
df.area.describe()

In [ ]:
df['class_id'].value_counts()

In [ ]:
pd.crosstab(df['class_id'],df['rad_id'])

In [ ]:
s = np.array(df['class_id'].value_counts())

In [ ]:
np.sum(s)-s[0]

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline


def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
fig=plt.figure(figsize=(12, 10))
columns = 2
rows = 2
for i in range(1, columns*rows +1):
    p = df.image_name[i]
    img = read_xray(p)
    print(img.shape)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img, 'gray')
plt.show()

def plot_class_id(class_id, col, row):
    """
    This function plots x-ray images of selected class. Pass class_id an integer from 0 to 14.
    The number of images to plot depends in col and row input which is integer.
    """
    fig=plt.figure(figsize=(12, 10))
    columns = col
    rows = row
    from random import sample
    ids = list(df.index[df.class_id == class_id])
    to_plot = sample(ids,row*col)
    counter = 1
    for i in to_plot:
        p = df.image_name[i]
        #print(p)
        img = read_xray(p)
        print(img.shape)
        fig.add_subplot(rows, columns, counter)
        plt.imshow(img, 'gray')
        counter +=1
    plt.show()

def plot_class_box(class_id, col, row):
    """
    This function plots x-ray images of selected class and box identifying abnormality. 
    Pass class_id an integer from 0 to 14.
    The number of images to plot depends in col and row input which is integer.
    """
    fig=plt.figure(figsize=(12, 10))
    columns = col
    rows = row
    from random import sample
    ids = list(df.index[df.class_id == class_id])
    to_plot = sample(ids,row*col)
    counter = 1
    for i in to_plot:
        p = df.image_name[i]
        #print(p)
        img = read_xray(p)
        print(img.shape)
        fig.add_subplot(rows, columns, counter)
        plt.imshow(img, 'gray')
        plt.gca().add_patch(Rectangle((df.x_max[i],df.y_max[i]),df.xd[i],df.yd[i],linewidth=1,edgecolor='r',facecolor='none'))
        counter +=1
    plt.show()

def plot_class_crop(class_id, col, row):
    """
    This function plots x-ray images of selected class and box identifying abnormality. 
    Pass class_id an integer from 0 to 14.
    The number of images to plot depends in col and row input which is integer.
    """
    fig=plt.figure(figsize=(12, 10))
    columns = col
    rows = row
    from random import sample
    ids = list(df.index[df.class_id == class_id])
    to_plot = sample(ids,row*col)
    counter = 1
    for i in to_plot:
        p = df.image_name[i]
        #print(p)
        img = read_xray(p)
        fig.add_subplot(rows, columns, counter)
        im = img[int(df.x_min[i]):int(df.x_max[i]),int(df.y_min[i]):int(df.y_max[i])]
        print(im.shape)
        plt.imshow(im, 'gray')
        counter +=1
    plt.show()

def plot_class_crop_head(class_id, col, row):
    """
    This function plots x-ray images of selected class and box identifying abnormality. 
    Pass class_id an integer from 0 to 14.
    The number of images to plot depends in col and row input which is integer.
    """
    columns = col
    rows = row
    from random import sample
    ids = list(df.index[df.class_id == class_id])
    to_plot = sample(ids,row*col)
    counter = 1
    for i in to_plot:
        p = df.image_name[i]
        #print(p)
        img = read_xray(p)
        im = img[int(df.x_min[i]):int(df.x_max[i]),int(df.y_min[i]):int(df.y_max[i])]
        print(im[0:10,0:10])

In [ ]:
def features(class_id, df):
    gb = df.groupby('class_id')
    indx = df[df.class_id == class_id].index
    df1 = df.iloc[indx,:]
    minareaindx = df1[df1.area == df1.area.min()].index
    out = {'dataframe': df1, 'minareaindx': minareaindx}
    return out

out = features(0,df)
out['minareaindx']
#out['dataframe'].head(1)

In [ ]:
#pip install opencv_python==3.4.2.16 
#pip install opencv-contrib-python==3.4.2.16

img = read_xray('../input/vinbigdata-chest-xray-abnormalities-detection/train/0108949daa13dc94634a7d650a05c0bb.dicom')
im = img[int(df.x_min[2]):int(df.x_max[2]),int(df.y_min[2]):int(df.y_max[2])]

out['dataframe'].loc[np.asscalar(out['minareaindx']),'image_name']

In [ ]:
def similar_array(df, class_id, min_area_index):
    k = np.asscalar(min_area_index)
    out = features(class_id,df)
    #print('Base image is {}'.format(out['dataframe'].loc[k,'image_name']))
    img_base = read_xray(out['dataframe'].loc[k,'image_name'])
    im = img_base[int(df.x_min[k]):int(df.x_max[k]),int(df.y_min[k]):int(df.y_max[k])]
    #plt.imshow(im, 'gray')
    nrow = im.shape[0]
    ncol = im.shape[1]
    img_index_oth = df[df.class_id == class_id].index[0:200]
    
    for index in img_index_oth:
        current_directory = os.getcwd()
        final_directory = os.path.join(current_directory, r'features'+str(class_id)+'_')
        if not os.path.exists(final_directory):
            os.makedirs(final_directory)
        #print(out['dataframe'].loc[index,'image_name'])
        img_trial = read_xray(out['dataframe'].loc[index,'image_name'])
        img_trial = img_trial[int(df.x_min[index]):int(df.x_max[index]),int(df.y_min[index]):int(df.y_max[index])]
        #plt.imshow(img_trial, 'gray')
        ssi = []
        idx = []
        for r in list(range(img_trial.shape[0]-nrow)):            #img_trial.shape[0]-1
            for c in list(range(img_trial.shape[1]-ncol)):          #img_trial.shape[1]-1
                cropped = img_trial[r:(nrow+r),c:(ncol+c)]
                idx.append((r,c))
                ssi.append(np.linalg.norm(im - cropped))
        #print('Done for {}'.format(out['dataframe'].loc[index,'image_name']))
        if len(ssi)>0:
            minssi_id = np.array(ssi).argmin()
            idx_min = idx[minssi_id]
            min_crop = img_trial[idx_min[0]:(nrow+idx_min[0]),idx_min[1]:(ncol+idx_min[1])]
            #cv2.imwrite("features"+str(class_id)+str(r)+str(c)+'.png', min_crop)
            #dct = {"minssi_id":minssi_id,"idx_min":idx_min,"base_shape":im.shape,
            #      "cropped.shape":cropped.shape,"fitted_crop":min_crop.shape}
            #print(final_directory+str(idx_min[0])+str(idx_min[1])+'.jpg')
            #to_dir.save("features"+str(class_id)+str(idx_min[0])+str(idx_min[1])+'.jpg')
            cv2.imwrite(final_directory+str(idx_min[0])+str(idx_min[1])+'.jpg', min_crop)
    #print(img_base, img_index_oth)
    #return ssi, idx

In [ ]:
#print(img.shape)
#print(list(range(img.shape[0] - im.shape[0])))
#class_id=12;r=2;c=25
#"features"+str(class_id)+str(r)+str(c)+'.jpg'

current_directory = os.getcwd()
class_id=12
final_directory = os.path.join(current_directory, r'features'+str(class_id))
final_directory

out = features(12,df)
similar_array(df,12,out['minareaindx'])

In [ ]:
for class_id in list(range(14)):
    print('working on class id = {}'.format(class_id))
    out = features(class_id,df)
    similar_array(df,class_id,out['minareaindx'])